# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [120]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202206.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202206.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [121]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[가자미수입]가자미수입,4.5kg,보통(3등),18500,수입 러시아\t,일반,20220601
1,[가자미수입]가자미수입,4.5kg,보통(3등),16000,수입 러시아\t,일반,20220601
2,[가자미수입]가자미수입,4.5kg,보통(3등),18500,수입 러시아\t,일반,20220601
3,[가자미수입]가자미수입,4.5kg,보통(3등),18500,수입 러시아\t,일반,20220601
4,[가지]가지,5kg,특(1등),11000,경남 진주시,일반,20220601


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072668 entries, 0 to 1072667
Data columns (total 7 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   품목명     1072668 non-null  object
 1   단위      1072668 non-null  object
 2   등급      1072668 non-null  object
 3   가격      1072668 non-null  int64 
 4   산지      1071216 non-null  object
 5   친환경구분   1072668 non-null  object
 6   입력일     1072668 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 57.3+ MB


In [123]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       1452
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [124]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [125]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [126]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [127]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [128]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [129]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,일반,20220601,3
1,[가자미수입]가자미수입,4.5kg,16000,수입 러시아\t,일반,20220601,3
2,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,일반,20220601,3
3,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,일반,20220601,3
4,[가지]가지,8kg,3500,경기 안성시,일반,20220601,3
...,...,...,...,...,...,...,...
1071211,[토마토]토마토(기타),5kg,5000,인천 강화군,일반,20220630,5
1071212,[토마토]토마토(기타),5kg,6000,인천 강화군,일반,20220630,5
1071213,[토마토]흑토마토,2kg,2500,강원,일반,20220630,5
1071214,[토마토]흑토마토,2kg,2500,강원,일반,20220630,5


### eco

In [130]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [131]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,normal,20220601,3
1,[가자미수입]가자미수입,4.5kg,16000,수입 러시아\t,normal,20220601,3
2,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,normal,20220601,3
3,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,normal,20220601,3
4,[가지]가지,8kg,3500,경기 안성시,normal,20220601,3
...,...,...,...,...,...,...,...
1071211,[토마토]토마토(기타),5kg,5000,인천 강화군,normal,20220630,5
1071212,[토마토]토마토(기타),5kg,6000,인천 강화군,normal,20220630,5
1071213,[토마토]흑토마토,2kg,2500,강원,normal,20220630,5
1071214,[토마토]흑토마토,2kg,2500,강원,normal,20220630,5


### reg_date

In [132]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [133]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [134]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,normal,2022-06-01,3
1,[가자미수입]가자미수입,4.5kg,16000,수입 러시아\t,normal,2022-06-01,3
2,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,normal,2022-06-01,3
3,[가자미수입]가자미수입,4.5kg,18500,수입 러시아\t,normal,2022-06-01,3
4,[가지]가지,8kg,3500,경기 안성시,normal,2022-06-01,3
...,...,...,...,...,...,...,...
1071211,[토마토]토마토(기타),5kg,5000,인천 강화군,normal,2022-06-30,5
1071212,[토마토]토마토(기타),5kg,6000,인천 강화군,normal,2022-06-30,5
1071213,[토마토]흑토마토,2kg,2500,강원,normal,2022-06-30,5
1071214,[토마토]흑토마토,2kg,2500,강원,normal,2022-06-30,5


### price

In [135]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [136]:
df2['scale'].unique()

array([    4.5 ,     8.  ,     5.  ,     8.01,    20.  ,    10.  ,
           3.  ,     2.  ,     0.25,     0.05,     0.07,     9.  ,
           0.8 ,     0.6 ,     2.2 ,     1.9 ,     1.8 ,     4.  ,
          21.5 ,     1.  ,     1.5 ,     1.4 ,     1.6 ,    15.  ,
           6.  ,     3.4 ,    12.  ,    23.  ,    11.  ,     0.5 ,
           7.5 ,    21.  ,    18.  ,     3.6 ,     2.5 ,    14.  ,
          16.  ,     3.5 ,     4.4 ,    11.5 ,    13.  ,     0.7 ,
           0.9 ,     7.  ,     2.8 ,    10.8 ,     5.4 ,     1.1 ,
           2.1 ,     5.1 ,    22.5 ,     1.7 ,     9.5 ,     1.3 ,
           3.2 ,     4.3 ,    24.  ,     1.2 ,     7.8 ,    22.  ,
           6.6 ,     8.5 ,    10.1 ,    25.  ,     2.16,     9.3 ,
           0.98,     9.2 ,    12.5 ,    10.7 ,     2.9 ,     3.8 ,
          19.  ,     2.4 ,     0.4 ,     4.8 ,    12.8 ,    76.5 ,
          77.5 ,    80.  ,    71.5 ,    73.  ,    75.  ,    70.5 ,
          11.4 ,     4.2 ,    68.5 ,    79.  ,    69.5 ,    81

In [137]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count    1071216.00000
mean       18313.91964
std        52880.36680
min            1.00000
25%         8000.00000
50%        13000.00000
75%        21000.00000
max     46315500.00000
Name: price, dtype: float64

In [138]:
df2 = df2.drop(df2[df2['price']<500].index)

In [139]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가자미수입]가자미수입,4.50000,18500,수입 러시아\t,normal,2022-06-01,3
1,[가자미수입]가자미수입,4.50000,16000,수입 러시아\t,normal,2022-06-01,3
2,[가자미수입]가자미수입,4.50000,18500,수입 러시아\t,normal,2022-06-01,3
3,[가자미수입]가자미수입,4.50000,18500,수입 러시아\t,normal,2022-06-01,3
4,[가지]가지,8.00000,3500,경기 안성시,normal,2022-06-01,3
...,...,...,...,...,...,...,...
1071211,[토마토]토마토(기타),5.00000,5000,인천 강화군,normal,2022-06-30,5
1071212,[토마토]토마토(기타),5.00000,6000,인천 강화군,normal,2022-06-30,5
1071213,[토마토]흑토마토,2.00000,2500,강원,normal,2022-06-30,5
1071214,[토마토]흑토마토,2.00000,2500,강원,normal,2022-06-30,5


In [140]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [141]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[참외]참외             60100
[오이]백다다기           40985
[참외]금싸라기(참외)       31324
[참외]금싸라기           25738
[깻잎]깻잎             24122
                   ...  
[가오리수입]가오리수입           1
[산딸기]산딸기 기타            1
[청어수입]청어수입             1
[기타엽채]기타엽채(청경채)        1
[복숭아]복숭아(수황)           1
Name: prd, Length: 1469, dtype: int64

In [142]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [143]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [144]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [145]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가죽나물 가지 간재미 갈치 감 감귤 감자 갑오징어 갓 강낭콩 개불 
개조개 갯가재 건고구마순 게 게지 고 고구마 고구마순 고둥 고들빼 고등어 고사리 고추 고추잎 곤달비 
곤드레나물 골뱅이 과실류 과일 과일류 과채 굴 귤 그레이프푸룻 근대 근채류 깻잎 꼬막 꼴뚜 꽁치 
꽈리고추 낙지 넙치 논우렁이 눈볼대 느 다래 다시마 닭새우 당근 대구 대구고니 대구머리 대구포 대추 
대파 도다리 돗나물 동부콩 동자개 동죽 두류 두릅 두리안 듀리안 딸 땅콩 라임 람부탄 레몬 
리치 마늘 만가닥 만감 맛 망고 망고스턴 망고스틴 매실 머위대 메 메론 명태 명태곤니 명태알 
명태포 모시조개 무 무순 무우 무화과 문어 문어바지락 미꾸라지 미나리 미더덕 민들레 민물새우 바나나 바지락 
박 밤 방울토마토 방풍 방풍나물 배 배추 백새우 백조 백합 뱀장어 버섯 버섯류 버찌 벗 
병어 보리 복분자 복숭아 봉지굴 부세 부지갱이 부추 북방대합조개 붉은고추 붕어 붕장어 브로커리 브로코리 블루베리 
비름 비트 빈스 빙어 사과 살구 삼 삼치 상추 새꼬막 새송이 새싹 새우 새우살 새조개 
샐러리 생강 생고추 세러리 셀러리 소라 속새 숙주나물 숫꽃게 시금치 시바 실파 쌈채류 쑥 쑥갓 
아귀 아보카도 아보카드 아스파라가스 아스파라거스 아욱 알 알배 앵두 양배추 양상추 양송이 양채 양채류 양파 
얼갈이 얼갈이배추 연근 연어 열무 염고등어 엽경채류 엽채 엽채류 오디 오렌지 오만둥이 오이 오징어 오징어다리 
오징어원양 오징어채 옥 완두콩 용과 우렁쉥이 우렁이 유채 임연 잉어 자두 자라 자몽 재첩 적근대 
적새우 적채 절단 조개살 죽순 쪽파 쭈꾸미 참나물 참다래 참외 참조 참죽나무순 채소 천도복숭아 청경채 
청어 체리 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 칼리후라워 케일 코라비 코코넛 콜라비 콩 콩나물 
키위 키조개 키조개날개 토마토 파 파세리 파인애플 파파야 파프리카 패류 팽이 팽이버섯 포도 포장바지락 표고 
표고버섯 풋고추 풋옥 피망 피뿔고동 피조개 학꽁치 한치

In [146]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [147]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [148]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [149]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
df2.loc[df2["prd"]=="대구알","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 로케트 -> 루꼴라로 변경
df2.loc[df2["prd"]=="로케트","prd"] = "루꼴라"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
df2.loc[df2["prd"]=="코다리명태","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
df2.loc[df2["prd"]=="벗찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채원양","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어살","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어몸통","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 꽃게 관련 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="꽃게살","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 식용허브 -> 허브로 변경
df2.loc[df2["prd"]=="식용허브","prd"] = "허브"
# 선인장열매 -> 선인장으로 변경
df2.loc[df2["prd"]=="선인장열매","prd"] = "선인장"
# 쌈추 -> 쌈채류로 변경
df2.loc[df2["prd"]=="쌈추","prd"] = "쌈채류"
# 덟은감 -> 감으로 변경
df2.loc[df2["prd"]=="덟은감","prd"] = "감"
# 피뿔고동 -> 고둥으로 변경
df2.loc[df2["prd"]=="피뿔고동","prd"] = "고둥"
# 꼬시래 -> 꼬시래기으로 변경
df2.loc[df2["prd"]=="꼬시래","prd"] = "꼬시래기"
# 연어살 -> 연어로 변경
df2.loc[df2["prd"]=="연어살","prd"] = "연어"
# 비단멍게 -> 멍게로 변경
df2.loc[df2["prd"]=="비단멍게","prd"] = "멍게"
# 물가자미 -> 가자미로 변경
df2.loc[df2["prd"]=="물가자미","prd"] = "가자미"
# 물메 -> 메기로 변경
df2.loc[df2["prd"]=="물메","prd"] = "메기"
# 붉은메 -> 메기로 변경
df2.loc[df2["prd"]=="붉은메","prd"] = "메기"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)
df2.drop(df2[df2['prd']=='어류원양'].index,inplace=True)
df2.drop(df2[df2['prd']=='동물'].index,inplace=True)

In [150]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array([], dtype=object)

In [151]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_4135/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [152]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [153]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [154]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,227.00000
mean,4713.93392
std,13640.33335
min,1.00000
25%,14.00000
50%,172.00000
75%,2406.00000
max,129039.00000


In [155]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [156]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

참외       129039
버섯        74863
오이        64719
토마토       56705
복숭아       54264
          ...  
재첩           64
개조개          64
참죽나무순        63
죽순           58
갈치           53
Name: prd, Length: 148, dtype: int64

In [157]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [158]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([    4.5 ,     5.  ,    18.  ,     3.  ,    19.  ,     8.  ,
           8.01,     0.6 ,     1.5 ,     7.  ,     5.01,     1.  ,
           0.4 ,     4.  ,     6.  ,    10.  ,    20.  ,    15.  ,
           0.5 ,     9.6 ,    10.4 ,     9.  ,     5.5 ,     1.2 ,
           0.8 ,    12.  ,     2.  ,    11.  ,    13.  ,    17.  ,
          14.  ,    21.  ,    25.  ,    23.  ,    16.  ,     7.5 ,
           3.2 ,     9.5 ,    12.5 ,     3.6 ,     2.6 ,     3.8 ,
           3.5 ,     2.2 ,     2.5 ,     0.2 ,     3.4 ,     2.7 ,
           2.1 ,     2.9 ,     3.9 ,    11.5 ,     3.7 ,     1.4 ,
           2.4 ,     2.8 ,     2.3 ,     4.2 ,     3.3 ,     1.9 ,
           6.5 ,     1.6 ,     4.3 ,    10.6 ,     5.4 ,     8.5 ,
           8.3 ,     1.7 ,     5.3 ,     0.75,     3.1 ,     9.4 ,
           5.2 ,     1.8 ,     6.6 ,   100.  ,     6.7 ,     5.7 ,
           8.6 ,     7.4 ,     4.6 ,     6.8 ,     5.6 ,    14.5 ,
           0.1 ,     7.6 ,     6.4 ,     0.25,     0.05,     0

In [159]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
81169,고추,100.00000,27000,전남 곡성군,normal,2022-06-24,1,270.00000
148192,당근,200.00000,32000,경남 김해시,normal,2022-06-27,1,160.00000
148194,당근,200.00000,38000,경남 김해시,normal,2022-06-27,1,190.00000
148208,당근,200.00000,23000,경남 김해시,normal,2022-06-27,1,115.00000
148209,당근,200.00000,23000,경남 김해시,normal,2022-06-27,1,115.00000
148210,당근,200.00000,23000,경남 김해시,normal,2022-06-27,1,115.00000
148211,당근,200.00000,27000,경남 김해시,normal,2022-06-27,1,135.00000
148212,당근,200.00000,23000,경남 김해시,normal,2022-06-27,1,115.00000
148213,당근,200.00000,23000,경남 김해시,normal,2022-06-27,1,115.00000
148230,당근,200.00000,23000,경남 김해시,normal,2022-06-27,1,115.00000


In [160]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]['prd'].unique()

array(['고추', '당근', '매실', '복숭아', '수박', '양파', '오이', '호박', '바나나'],
      dtype=object)

In [161]:
df3[(df3['prd']=='바나나')&(df3['scale']>=10)]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1029710,바나나,13.00000,26000,필리핀,normal,2022-06-01,1,2000.00000
1029711,바나나,13.00000,26000,필리핀,normal,2022-06-01,1,2000.00000
1029712,바나나,13.00000,23000,필리핀,normal,2022-06-01,1,1769.23000
1029713,바나나,13.00000,21000,필리핀,normal,2022-06-01,1,1615.38000
1029714,바나나,13.00000,20000,필리핀,normal,2022-06-01,1,1538.46000
...,...,...,...,...,...,...,...,...
1038079,바나나,13.00000,14000,수입산 필리핀,normal,2022-06-30,9,1076.92000
1038080,바나나,13.00000,10000,수입산 과테말라,normal,2022-06-30,9,769.23000
1038081,바나나,13.00000,11000,수입산 베트남,normal,2022-06-30,9,846.15000
1038082,바나나,13.00000,14000,수입산 베트남,normal,2022-06-30,9,1076.92000


In [162]:
# 스케일 오표기된 항목 수정
# 양파,호박 정상 거래
# 이상치 항목 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='양파')&(df3['prd']!='호박')].index)


In [163]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
501608,양파,1000.00000,800000,경북 김천시,normal,2022-06-18,1,800.00000
504824,양파,1000.00000,800000,경북 김천시,normal,2022-06-27,1,800.00000
821746,호박,58.00000,10000,강원 춘천시,normal,2022-06-09,1,172.41000
821750,호박,58.00000,11000,강원 춘천시,normal,2022-06-09,1,189.66000
821753,호박,58.00000,6000,강원 춘천시,normal,2022-06-09,1,103.45000
844193,호박,58.00000,4500,강원 춘천시,normal,2022-06-09,2,77.59000


### origin

In [164]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['수입 러시아\t', '전북 군산\t', '수입 미국\t', '경기 안성시', '경기 여주시', '경기 용인시',
       '경남 창녕군', '경북 성주군', '경기 이천시', '경기 하남시', '경남 밀양시', '경남 창원시 의창구',
       '경기도 하남시', '경기도 여주시 능서면', '광주 광산구', '경기도 여주시 점동면', '경남 진주시',
       '경기 여주군', '광주시 광산구', '경기 광주시', '충남 아산시', '경기 여주시 여주읍', '경북 영주시',
       '경기 고양시', '충남 예산군', '서울 강동구', '경남 창원시', '강원 춘천시', '강원 평창군',
       '경기 양평군', '경남 김해시', '충남 천안시', '경남 고성군', '경남 의령군', '전북 전주시',
       '경남 거제시', '경기 안성시 금광면', '경기 화성시', '성남시 중원구', '강원 홍천군', '경기 평택시',
       '경기 안성시 양성면', '충북 충주시', '강원도 철원', '경기 파주시', '강원도 홍천', '강원 횡성군',
       '경기 고양시 일산', '경기도 평택시', '경남 진주시 금곡면', '강원 양구군', '전남 나주시', '전남 무안군',
       '강원 철원군', '충남 보령시', '강원도 철원군', '강원 화천군', '전라북도 순창군', '수입 기타\t',
       '수입 세네갈\t', '제주 제주도\t', '국내산\t', '수입 오만\t', '제주 서귀포시', '제주 제주시',
       '제주 서귀포시(2)', '제주도 서귀포시', '제주도 제주시', '제주', '전북 남원시', '제주 서귀포',
       '전남 보성군', '서울시 강서구', '충남 당진시', '충남 부여군', '충남 서산시', '경북 구미시',
       '충남 서산시 부석면', '전라북도 부안군', '충남 논산시', '전북 김제시', '충북 진천군 초평면',
       '전북 부안군', '

In [165]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,가자미,4.50000,18500,수입 러시아\t,normal,2022-06-01,3,4111.11000,수입,러시아\t,NaN,NaN
1,가자미,4.50000,16000,수입 러시아\t,normal,2022-06-01,3,3555.56000,수입,러시아\t,NaN,NaN
2,가자미,4.50000,18500,수입 러시아\t,normal,2022-06-01,3,4111.11000,수입,러시아\t,NaN,NaN
3,가자미,4.50000,18500,수입 러시아\t,normal,2022-06-01,3,4111.11000,수입,러시아\t,NaN,NaN
4,가자미,4.50000,14000,수입 러시아\t,normal,2022-06-03,3,3111.11000,수입,러시아\t,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1069190,천도복숭아,5.00000,6000,경북 경산시,normal,2022-06-29,1,1200.00000,경북,경산시,NaN,NaN
1069191,천도복숭아,10.00000,4500,경북 경산시,normal,2022-06-23,9,450.00000,경북,경산시,NaN,NaN
1069192,천도복숭아,5.00000,3500,경북 경산시,normal,2022-06-24,9,700.00000,경북,경산시,NaN,NaN
1069193,천도복숭아,5.00000,6500,경북 경산시,normal,2022-06-27,9,1300.00000,경북,경산시,NaN,NaN


In [166]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [167]:
df4['state'].unique()

array(['수입', '전북', '경기', '경남', '경북', '경기도', '광주', '광주시', '충남', '서울', '강원',
       '성남시', '충북', '강원도', '전남', '전라북도', '제주', '국내산\t', '제주도', '서울시',
       '인천시', '청주시', '인천', '대구', '호주', '수입산', '세종시', '대전', '부산', '타이',
       '태국', '베트남', '필리핀', '멕시코', '미국', '우즈베키스탄', '서울특별시', '중국', '뉴질랜드',
       '충청남도', '충북청주시상당구', '세종', '대전시', '제주시', '대구시', '부산시', '경북/문경시',
       '전주시', '포클랜드\t', '수원시', '경상남도', '전라남도', '기타외국', '인천광역시', '칠레',
       '페루', '오스트레일', '경기고양시일산구', '전북전주시완산구', '캘리포니아', '대만', '부천시',
       '에콰도르', '콜롬비아', '에쿠아도르', '과테말라', '코스타리카', '우즈베크', '우크라이나', '가락동',
       '스페인', '이스라엘', '남아프리카', '남아프리카공화국'], dtype=object)

In [168]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [169]:
df4["state"].unique()

array(['수입', '전북', '경기', '경남', '경북', '광주', '충남', '서울', '강원', '성남', '충북',
       '전남', '전라북', '제주', '국내산', '인천', '청주', '대구', '호주', '수입산', '세종',
       '대전', '부산', '타이', '태국', '베트남', '필리핀', '멕', '미국', '우즈베키스탄', '서울특별',
       '중국', '뉴질랜드', '충청남', '충북청주', '경북/문경', '전주', '포클랜드', '수원', '경상남',
       '전라남', '기타외국', '인천광역', '칠레', '페루', '오스트레일', '경기고양', '전북전주',
       '캘리포니아', '대만', '부천', '에콰', '콜롬비아', '에쿠아', '과테말라', '코스타리카', '우즈베크',
       '우크라이나', '가락동', '스페인', '이스라엘', '남아프리카', '남아프리카공화국'], dtype=object)

In [170]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="부산광역","city"] = "부산"
df4.loc[df4["state"]=="부산광역","state"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","state"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="서귀포","city"] = "제주"
df4.loc[df4["state"]=="서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"
df4.loc[df4["state"]=="전남/무안군","city"] = "전남"
df4.loc[df4["state"]=="전남/무안군","state"] = "무안"
df4.loc[df4["state"]=="경북/구미","city"] = "구미"
df4.loc[df4["state"]=="경북/구미","state"] = "경북"
df4.loc[df4["state"]=="경북/상주","city"] = "상주"
df4.loc[df4["state"]=="경북/상주","state"] = "경북"
df4.loc[df4["state"]=="경북/영천","city"] = "영천"
df4.loc[df4["state"]=="경북/영천","state"] = "경북"
df4.loc[df4["state"]=="경남/합천군","city"] = "합천"
df4.loc[df4["state"]=="경남/합천군","state"] = "경남"
df4.loc[df4["state"]=="전남/고흥군","city"] = "고흥"
df4.loc[df4["state"]=="전남/고흥군","state"] = "전남"
df4.loc[df4["state"]=="무안","state"] = "전남"
df4.loc[df4["state"]=="서해안","city"] = "미상"
df4.loc[df4["state"]=="서해안","state"] = "국산"
df4.loc[df4["state"]=="남해안","city"] = "미상"
df4.loc[df4["state"]=="남해안","state"] = "국산"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[df4["state"]=="캄보디아","city"] = "캄보디아"
df4.loc[df4["state"]=="캄보디아","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"
df4.loc[df4["state"]=="그리스","city"] = "그리스"
df4.loc[df4["state"]=="그리스","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"
df4.drop(df4[df4["state"]=="미등록"].index,inplace=True)

In [171]:
df4['state'].unique()

array(['수입', '전북', '경기', '경남', '경북', '광주', '충남', '서울', '강원', '충북', '전남',
       '제주', '국산', '인천', '대구', '세종', '대전', '부산'], dtype=object)

In [172]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [173]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [174]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [175]:
df4['city'].unique()

array(['러', '산', '미국', '안성', '여주', '용인', '창녕', '성주', '이천', '하남', '밀양',
       '창원', '광산구', '진주', '광주', '아산', '영주', '고양', '예산', '강동구', '춘천', '평창',
       '양평', '김해', '천안', '고성', '의령', '전주', '거제', '화성', '성남', '홍천', '평택',
       '충주', '철원', '파주', '횡성', '양구', '나주', '무안', '보령', '화천', '순창', '기타',
       '세네갈', '제주', '미상', '오만', '남원', '보성', '강서구', '당진', '부여', '서산', '구미',
       '부안', '논산', '김제', '진천', '홍성', '서천', '고창', '태안', '곡성', '강화', '김포',
       '구례', '익산', '고흥', '금산', '양주', '괴산', '해남', '예천', '청원', '장흥', '칠곡',
       '청주', '순천', '영광', '공주', '강릉', '원주', '사천', '김천', '광명', '상주', '음성',
       '옥천', '포천', '계양구', '안동', '대구', '호주', '중구', '임실', '연천', '청양', '송파구',
       '영암', '화순', '세종', '광양', '영동', '제천', '완도', '고령', '여수', '장항(서천)',
       '베트남', '구리', '남양주', '통영', '중국', '완주', '삼천포', '인천', '노르웨이', '네덜란드',
       '프랑스', '남구', '함안', '하동', '인제', '청도', '대전', '도봉구', '부산', '서구', '정선',
       '태국', '남해', '북구', '영덕', '필리핀', '담양', '멕', '우즈베키스탄', '경산', '진도',
       '서울', '중랑구', '동대문구', '흥', '의왕', '영양', '양천

In [176]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="익","city"] = "익산"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="주문진","city"] = "강릉"
df4.loc[df4["city"]=="울릉도","city"] = "울릉"
df4.loc[df4["city"]=="거제(장승포)","city"] = "거제"
df4.loc[(df4["city"]=="완")&(df4["state"]=="전북"),"city"] = "완주"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="신"),"city"] = "신안"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="장"),"city"] = "장수"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="진"),"city"] = "진도"
df4.loc[(df4["state"]=="충남")&(df4["city"]=="부"),"city"] = "부여"
df4.loc[(df4["state"]=="경남")&(df4["city"]=="의"),"city"] = "의령"
df4.loc[(df4["state"]=="전남")&(df4["city"]=="함"),"city"] = "함평"
# 전라남도 장 -> 장흥, 장성 두개 있으므로 구분 불가, 삭제
df4.drop(df4[(df4["city"]=="장")&(df4["state"]=="전남")].index,inplace=True)

## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"
df4.loc[df4["city"]=="용산구","city"] = "서울"
df4.loc[df4["city"]=="광진구","city"] = "서울"
df4.loc[df4["city"]=="종로구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
df4.loc[df4["city"]=="사우디","city"] = "사우디아라비아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [177]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array([], dtype=object)

In [178]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_4135/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [179]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [180]:
cities.to_csv('../cities.csv')

In [181]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [182]:
df5.to_csv('../data_proceed/202206_proceed.csv')